In [1]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [3]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\KS\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [4]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\KS\20161108__ks__general__precinct.csv


In [5]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\KS\20160802__ks__primary__edwards__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\KS\20160802__ks__primary__gray__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\KS\20160802__ks__primary__logan__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\KS\20160802__ks__primary__riley__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\KS\20160802__ks__primary__wabaunsee__precinct.csv


In [7]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
            
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()
        df["county"] = df["county"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]   
            df = df[~df['precinct'].str.upper().str.contains(r"\w*ABSENTEE\w*", na=False)]   

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]               
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENT"]
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
pri_combined_df


,county,precinct,office,district,party,candidate,votes


In [38]:
## Cleaning precinct, county name
# Fixing Cubbler
# pri_combined_df.loc[
#     ( 
#         pri_combined_df["county"].str.contains("\w*F120\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*STONE\w*", case=False, na=False)|
#         pri_combined_df["county"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "county")
# ] = "STONE"

In [39]:
# Checking any suspicious precinct, county
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].isna() |
    pri_combined_df["precinct"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["precinct"].str.strip().str.upper().isin(["NONE", "NAN"]) |
    pri_combined_df["county"].isna() |
    pri_combined_df["county"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["county"].str.strip().str.upper().isin(["NONE", "NAN"])
]

In [40]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,BREATHITTA101,NaN,Martin J. O'MALLEY,0
1,BREATHITTA102,NaN,Martin J. O'MALLEY,4
2,BREATHITTA103,NaN,Martin J. O'MALLEY,6
3,BREATHITTA104,NaN,Martin J. O'MALLEY,5
4,BREATHITTA105,NaN,Martin J. O'MALLEY,5
...,...,...,...,...
1890,ROCKCASTLEE102,NaN,"""UNCOMMITTED""",2
1891,ROCKCASTLEE103,NaN,"""UNCOMMITTED""",0
1892,ROCKCASTLEE104,NaN,"""UNCOMMITTED""",1
1893,ROCKCASTLEE105,NaN,"""UNCOMMITTED""",0


In [41]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Martin J. O'MALLEY          379
Bernard "Bernie" SANDERS    379
Hillary CLINTON             379
Roque Rocky De La FUENTE    379
"UNCOMMITTED"               379
Name: count, dtype: int64

In [42]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"Blank Votes", r"All Others",r"Total Votes Cast",r"No Preference",
    r"UNCOMMITTED","WRITE-IN"
]

pattern = "|".join(unwanted_keywords)

# # Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

#fIXING DE LA FUENTE
# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("rocky.*fuente", case=False, na=False) |
#         primary_data["candidate"].str.contains("LA FUENTE", case=False, na=False) |
#         primary_data["candidate"].str.contains("\w*Fuque\w*", case=False, na=False)
#     ),
#     "candidate"
# ] = "LA FUENTE"


# primary_data.loc[:, "candidate"] = primary_data["candidate"].replace({
#     "Donald I. Trump": "Donald J. Trump",
# })

primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Christie
primary_data.loc[
    primary_data["candidate"].str.contains("CHRISTLE", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing KASICH
primary_data.loc[
    primary_data["candidate"].str.contains("KAISCH", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("O'MALLEY", case=False, na=False),
    "candidate"
] = "O'MAILEY"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("PATAKL", case=False, na=False),
    "candidate"
] = "PATAKI"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("RUBLO", case=False, na=False),
    "candidate"
] = "RUBIO"

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

## SPLITTINNG BY ,
# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\214168396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\214168396.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\214168396.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy o

candidate
O'MAILEY    379
SANDERS     379
CLINTON     379
FUENTE      379
Name: count, dtype: int64

In [43]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
NaN    1516
Name: count, dtype: int64

In [44]:


primary_data["party"].value_counts(dropna=False)


party
NaN    1516
Name: count, dtype: int64

In [45]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "PRESIDENT REPUBLICAN PARTY": "REP",
        "PRESIDENT DEMOCRATIC PARTY": "DEM",
        "DEMOCRATIC PARTY": "DEM",
        "REPUBLICAN PARTY": "REP",
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"DEM",
        "NAN":"DEM"
    })
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\3664840311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\3664840311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\3664840311.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

party
DEM    1516
Name: count, dtype: int64

In [46]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_O'MAILEY,pri_dem_SANDERS
0,BREATHITTA101,29,1,0,31
1,BREATHITTA102,49,3,4,98
2,BREATHITTA103,90,4,6,159
3,BREATHITTA104,71,1,5,109
4,BREATHITTA105,57,3,5,63
...,...,...,...,...,...
374,ROCKCASTLEE101,22,0,0,15
375,ROCKCASTLEE102,10,0,0,5
376,ROCKCASTLEE103,14,0,0,4
377,ROCKCASTLEE104,3,0,0,8


In [47]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]      
            
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"].str.upper() == "PRESIDENT"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
general_files


['C:\\Huy Phan\\College\\VoterTurnout\\data\\KY\\20161108__ky__general__precinct.csv']

In [48]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,ADAIRA102,REP,Donald J. TRUMP / Michael R. PENCE,215
1,ADAIRA104,REP,Donald J. TRUMP / Michael R. PENCE,359
2,ADAIRA105,REP,Donald J. TRUMP / Michael R. PENCE,288
3,ADAIRAB,REP,Donald J. TRUMP / Michael R. PENCE,436
4,ADAIRB102,REP,Donald J. TRUMP / Michael R. PENCE,422
...,...,...,...,...
27724,HARTPRICEVILLE,DEM,Hillary Clinton,63
27725,HARTPRICEVILLE,LIB,Gary Johnson,6
27726,HARTPRICEVILLE,DEM,Rocky Roque DE LA FUENTE,0
27727,HARTPRICEVILLE,GRN,Jill Stein,3


In [49]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
Donald J. TRUMP / Michael R. PENCE                3562
Gary JOHNSON / Bill WELD                          3562
Hillary Rodham CLINTON / Timothy Michael KAINE    3562
Jill STEIN / Ajamu BARAKA                         3562
Rocky Roque DE LA FUENTE / Michael STEINBERG      3562
Evan McMULLIN / Nathan JOHNSON                    3562
Hillary Clinton                                    941
Gary Johnson                                       941
Jill Stein                                         941
Evan McMullin                                      941
Rocky De La Fuente                                 905
Donald Trump                                       860
Write-In                                           642
Donald J. Trump                                     81
Rocky Roque DE LA FUENTE                            36
DE LA FUENTE/STEINBERG                              10
STEIN/BARAKA                                        10
MCMULL/JOHNSON                                      10


In [51]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"NAN",
    r"\w*WRITE[\s-]\w*",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN"

]

# Fixing FUENTE
general_data.loc[
    general_data["candidate"].str.contains("\w*Fuenta\w*", case=False, na=False),
    "candidate"
] = "FUENTE"
pattern = "|".join(unwanted_keywords)

# # FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# Fixing Darrell Castle"
general_data.loc[
    general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
    "candidate"
] = "CASTLE"

# general_data.loc[
#     general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
#     "candidate"
# ] = "MATUREN"

# Fixing mixed president + vice_president by /
general_data["candidate"] = general_data["candidate"].str.extract(r"^([^/&]+)").iloc[:, 0].str.strip()

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# # Fixing the , candidate
# # general_data["candidate"] = (
# #     general_data["candidate"].str.split(",")
# #     .str[0]
# #     .str.strip()
# #     )




# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# # Fixing Kotlikoff
# general_data.loc[
#     general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
#     "candidate"
# ] = "KOTLIKOFF"

# # Fixing Valdivia
# general_data.loc[
#     general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
#     "candidate"
# ] = "VALDIVIA"

# # Fixing HOEFLING
# general_data.loc[
#     general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
#     "candidate"
# ] = "HOEFLING"


# # Fixing McMullin
general_data.loc[
    general_data["candidate"].str.contains("MCMULL", case=False, na=False) ,
    "candidate"
] = "MCMULLIN"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21788\2626623481.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


candidate
TRUMP       4513
MCMULLIN    4513
JOHNSON     4513
CLINTON     4513
STEIN       4513
FUENTE      4513
CASTLE         8
HOEFLING       1
Name: count, dtype: int64

In [52]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
DEM           9026
REP           4513
LIB           4513
GRN           4513
IND           4503
NaN             10
CON              8
Write - in       1
Name: count, dtype: int64

In [53]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
general_data["party"] = general_data.apply(
    lambda row: fill_party_from_data(row, general_data),
    axis=1
)

# # # Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
# general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
DEM           9026
REP           4513
IND           4513
LIB           4513
GRN           4513
CON              8
Write - in       1
Name: count, dtype: int64

In [55]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "AMERICAN DELTA":"AMD",
        "PROHIBITION":"PRO",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "WRITE - IN":"IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "INDEPENDENT":"IND",
        "(WRITE-IN)":"IND",
        "SOCIALISM & LIBERATION": "SOL",
        "SOCIALIST WORKERS": "SOW"
    })
)

general_data["party"].value_counts(dropna=False)

party
DEM    9026
IND    4514
REP    4513
LIB    4513
GRN    4513
CON       8
Name: count, dtype: int64

In [56]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [57]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_con_CASTLE,gen_dem_CLINTON,gen_dem_FUENTE,gen_grn_STEIN,gen_ind_HOEFLING,gen_ind_MCMULLIN,gen_lib_JOHNSON,gen_rep_TRUMP
0,ADAIRA102,0,10,0,3,0,2,1,215
1,ADAIRA104,0,28,0,3,0,3,6,359
2,ADAIRA105,0,30,1,0,0,2,2,288
3,ADAIRAB,0,97,0,0,0,8,9,436
4,ADAIRB102,0,148,1,6,0,9,14,422
...,...,...,...,...,...,...,...,...,...
3945,WOODFORDG102,0,224,0,4,0,15,31,413
3946,WOODFORDG103,0,183,1,6,0,8,23,416
3947,WOODFORDH101,0,155,0,5,0,7,15,352
3948,WOODFORDH102,0,221,1,6,0,15,17,369


In [58]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts

combined


candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_O'MAILEY,pri_dem_SANDERS,gen_con_CASTLE,gen_dem_CLINTON,gen_dem_FUENTE,gen_grn_STEIN,gen_ind_HOEFLING,gen_ind_MCMULLIN,gen_lib_JOHNSON,gen_rep_TRUMP,dem_primary_total,general_total
0,BREATHITTA101,29,1,0,31,0,31,0,0,0,0,1,83,61,115
1,BREATHITTA102,49,3,4,98,0,58,0,2,0,1,6,257,154,324
2,BREATHITTA103,90,4,6,159,0,102,2,2,0,3,9,407,259,525
3,BREATHITTA104,71,1,5,109,0,90,1,3,0,6,7,208,186,315
4,BREATHITTA105,57,3,5,63,0,64,0,1,0,1,3,159,128,228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,ROCKCASTLEE101,22,0,0,15,0,60,0,0,0,5,5,334,37,404
367,ROCKCASTLEE102,10,0,0,5,0,38,0,0,0,7,1,303,15,349
368,ROCKCASTLEE103,14,0,0,4,0,27,0,2,0,1,0,224,18,254
369,ROCKCASTLEE104,3,0,0,8,0,16,0,0,0,0,1,61,11,78


In [59]:
combined.to_csv("KY.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("KY_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("KY_general_filtered.csv", index=False)

In [60]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 379, general: 3950, combined: 371


In [61]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 18, general: 120


In [62]:
pri_combined_df['precinct'].unique()

array(['BREATHITTA101', 'BREATHITTA102', 'BREATHITTA103', 'BREATHITTA104',
       'BREATHITTA105', 'BREATHITTB101', 'BREATHITTB102', 'BREATHITTB103',
       'BREATHITTB104', 'BREATHITTB105', 'BREATHITTC101', 'BREATHITTC102',
       'BREATHITTC103', 'BREATHITTC104', 'BREATHITTC105', 'BREATHITTD101',
       'BREATHITTD102', 'BREATHITTD103', 'BREATHITTD104', 'BREATHITTD105',
       'BREATHITTD106', 'CLAYA101', 'CLAYA106', 'CLAYA107', 'CLAYB101',
       'CLAYB103', 'CLAYB104', 'CLAYC101', 'CLAYC102', 'CLAYC103',
       'CLAYC104', 'CLAYD102', 'CLAYD103', 'CLAYD104', 'CLAYE101',
       'CLAYE102', 'CLAYE103', 'CLAYE104', 'CLAYF101', 'CLAYF102',
       'CLAYF104', 'CLINTONA101', 'CLINTONA104', 'CLINTONB101',
       'CLINTONB102', 'CLINTONC101', 'CLINTONC102', 'CLINTONC103',
       'CLINTOND101', 'CLINTOND102', 'CLINTONE101', 'CLINTONE102',
       'CLINTONF101', 'CLINTONF102', 'ELLIOTTA101', 'ELLIOTTB101',
       'ELLIOTTC101', 'ELLIOTTD101', 'ELLIOTTE101', 'ELLIOTTF101',
       'ELLIOTTG101'

In [63]:
gen_combined_df['precinct'].unique()

array(['ADAIRA102', 'ADAIRA104', 'ADAIRA105', ..., 'HARTCUB RUN',
       'HARTTOTAL', 'HARTPRICEVILLE'], dtype=object)

In [64]:
primary_counties = set(pri_combined_df['precinct'].dropna().str.strip().str.upper())
general_counties = set(gen_combined_df['precinct'].dropna().str.strip().str.upper())

diff = primary_counties - general_counties
print(f"Counties in primary but not in general: {len(diff)}")
print(diff)


Counties in primary but not in general: 8
{'ELLIOTTF101', 'ELLIOTTC101', 'ELLIOTTA101', 'ELLIOTTB101', 'ELLIOTTG101', 'ELLIOTTE101', 'ROCKCASTLEABSE', 'ELLIOTTD101'}
